We use [`sympy`](https://www.sympy.org/en/index.html) for deriving or re-checking symbolic derivations
of the thesis. 

Be sure to have sympy version 1.23 or higher installed before running this notebook!


In [4]:
# uncomment cell below when you don't have sympy!
# 👇
# !pip install sympy
import sympy as smp
from sympy import Matrix, Symbol
from functools import reduce
from typing import Callable

In [5]:
x1, x2, x3, x4, x5, x6 = smp.symbols("x_1 x_2 x_3 x_4 x_5 x_6", real=True)

---
### Angle Embedding


Rotation gates

For the mathematical definition we refer to subsection "Quantum Computation" of the main text.

In [6]:
def rx(Θ: Symbol) -> Matrix:
    """Rotation gate around x-axis.

    Parameters
    ----------
    Θ : Symbol
        Rotation angle.

    Returns
    -------
    Matrix
        Matrix representation of rotation about the x-axis
        by an angle Θ.
    """
    return Matrix(
        [[smp.cos(Θ / 2), -1j * smp.sin(Θ / 2)], [-1j * smp.sin(Θ / 2), smp.cos(Θ / 2)]]
    )


def ry(Θ: Symbol) -> Matrix:
    """Rotation gate around y-axis.

    Parameters
    ----------
    Θ : Symbol
        Rotation angle.

    Returns
    -------
    Matrix
        Matrix representation of rotation about the y-axis
        by an angle Θ.
    """
    return Matrix([[smp.cos(Θ / 2), -smp.sin(Θ / 2)], [smp.sin(Θ / 2), smp.cos(Θ / 2)]])


def rz(Θ: Symbol) -> Matrix:
    """Rotation gate around z-axis.

    Parameters
    ----------
    Θ : Symbol
        Rotation angle.

    Returns
    -------
    Matrix
        Matrix representation of rotation about the z-axis
        by an angle Θ.
    """
    return Matrix([[smp.exp(-1j * Θ / 2), 0], [0, smp.exp(1j * Θ / 2)]])

Fidelity (is defined in the quantum machine learning section of the thesis): 

In [7]:
def dagger(Ψ: Matrix) -> Matrix:
    """Dagger operation for operators and 
    state vectors.

    Returns
    -------
    Matrix
        Transposed Matrix/Vector, where additionally 
        each element has been complex-conjugated.
    """
    return Ψ.conjugate().T

def fidelity(Ψ: Matrix, ϕ: Matrix) -> float:
    return smp.Abs(dagger(Ψ) @ ϕ) ** 2

We can easily re-check the expression we obtain for Angle embedding when using 
$\sigma = \left\{ X, Y \right\}$ and a vector $\bm{x} \in \mathcal{X}$ that has only one feature $d =1$:

In [8]:
num_qubits = 3
psi_init = smp.zeros(2 ** num_qubits, 1)
psi_init[0, 0] = 1.0

v1 = Matrix([[x1, x2, x3]]).T
v2 = Matrix([[x4, x5, x6]]).T

def angle(v: Matrix, rotation: Callable) -> Matrix:
    v = v.tolist()
    flattened_v = [item for row in v for item in row]
    return reduce(smp.kronecker_product, (rotation(x_i) for x_i in flattened_v))

psi_v1 = angle(v1, rx) @ psi_init
psi_v2 = angle(v2, rx) @ psi_init

In [9]:
fidelity(psi_v1, psi_v2)[0].simplify()

1.0*cos(x_1/2 - x_4/2)**2*cos(x_2/2 - x_5/2)**2*cos(x_3/2 - x_6/2)**2

In [10]:
psi_v1 = angle(v1, ry) @ psi_init
psi_v2 = angle(v2, ry) @ psi_init

fidelity(psi_v1, psi_v2)[0].simplify()

1.0*cos(x_1/2 - x_4/2)**2*cos(x_2/2 - x_5/2)**2*cos(x_3/2 - x_6/2)**2

---
### IQPEmbedding

Single hadamard gate and projector (see section for quantum computation for the former, section quantum theory for the latter):

In [11]:
def h() -> Matrix:
    """Hadamard gate

    Returns
    -------
    Matrix
        Matrix representation of Hadamard gate.
    """
    return (1 / smp.sqrt(2)) * Matrix([[1.0, 1.0], [1.0, -1.0]])

def projector(i: int, j: int, dim: int = 1) -> Matrix:
    """
    Creates projector onto the basis specified by indices i and j 
    in a Hilbert space of dimension 2^dim.

    Parameters
    ----------
    i : int
        Row index for the projector matrix element to set to 1.
    j : int
        Column index for the projector matrix element to set to 1.
    dim : int, optional
        Logarithmic dimension of the Hilbert space. The resulting matrix 
        will be of size 2^dim x 2^dim. Default is 1.

    Returns
    -------
    Matrix
        A square matrix of size 2^dim x 2^dim with a 1.0 at position (i, j) 
        and 0 elsewhere.
    """
    proj = smp.zeros(2 ** dim)
    proj[i, j] = 1.0
    return proj


In [12]:
def cz():
    return smp.kronecker_product(projector(0, 0), smp.eye(2)) + smp.kronecker_product(
        projector(1, 1), 1j * rx(smp.pi)
    )

In [13]:
v1 = Matrix([[x1, x2]]).T
v2 = Matrix([[x3, x4]]).T
num_qubits = len(v1)
psi_init = smp.zeros(2 ** num_qubits, 1)
psi_init[0] = 1.0

In [14]:
def u_iqp(v: Matrix) -> Matrix:
    h_wall = reduce(smp.kronecker_product, tuple([h()] * len(v)))
    rotations = smp.kronecker_product(rz(v[0]), rz(v[1]))
    entanglers = cz() @ rotations @ cz()
    return entanglers @ rotations @ h_wall 

In [15]:
psi_v1 = u_iqp(v1) @ u_iqp(v1) @ psi_init
psi_v2 = u_iqp(v2) @ u_iqp(v2) @ psi_init

In [17]:
fidelity(psi_v1, psi_v2)[0].simplify()

(-0.0078125*(1 - exp(2.0*I*x_2))*exp(1.0*I*x_2)*exp(I*(46.0*x_1 + 46.0*x_2 + 48.0*x_3 + 42.0*x_4)) - 0.00390625*(1 - exp(4.0*I*x_2))*exp(I*(48.0*x_1 + 46.0*x_2 + 48.0*x_3 + 42.0*x_4)) - 0.0078125*(1 - exp(2.0*I*x_4))*exp(1.0*I*x_4)*exp(I*(48.0*x_1 + 42.0*x_2 + 46.0*x_3 + 46.0*x_4)) - 0.00390625*(1 - exp(4.0*I*x_4))*exp(I*(48.0*x_1 + 42.0*x_2 + 48.0*x_3 + 46.0*x_4)) + 0.0078125*(-exp(2.0*I*x_1) + exp(I*(2.0*x_1 + 4.0*x_4)))*exp(I*(48.0*x_1 + 43.0*x_2 + 48.0*x_3 + 46.0*x_4)) + 0.0078125*(exp(2.0*I*x_1) - exp(2.0*I*(x_1 + x_4)))*exp(I*(48.0*x_1 + 43.0*x_2 + 44.0*x_3 + 46.0*x_4)) + 0.015625*(-exp(4.0*I*x_1) + exp(I*(4.0*x_1 + 2.0*x_4)))*exp(I*(48.0*x_1 + 44.0*x_2 + 44.0*x_3 + 46.0*x_4)) + 0.0078125*(exp(4.0*I*x_1) + 1)*exp(I*(48.0*x_1 + 44.0*x_2 + 46.0*x_3 + 45.0*x_4)) - 0.0078125*(exp(1.0*I*x_2) + exp(I*(3.0*x_2 + 4.0*x_3)))*exp(I*(46.0*x_1 + 46.0*x_2 + 48.0*x_3 + 44.0*x_4)) + 0.0078125*(-exp(2.0*I*x_3) + exp(I*(4.0*x_2 + 2.0*x_3)))*exp(I*(48.0*x_1 + 46.0*x_2 + 48.0*x_3 + 43.0*x_4)) + 0.0